In [ ]:
pip install prompt-learner --upgrade

In [ ]:
from prompt_learner.tasks.classification import ClassificationTask
from prompt_learner.examples.example import Example
from prompt_learner.selectors.random_sampler import RandomSampler
from prompt_learner.selectors.diverse_sampler import DiverseSampler
from prompt_learner.selectors.stratified_sampler import StratifiedSampler
from prompt_learner.prompts.prompt import Prompt
from prompt_learner.templates.markdown import MarkdownTemplate
from prompt_learner.templates.xml import XmlTemplate
from prompt_learner.adapters.ollama_local import OllamaLocal
from prompt_learner.adapters.anthropic import Anthropic
from prompt_learner.adapters.llama import Llama
from prompt_learner.adapters.openai import OpenAI
from prompt_learner.evals.metrics.accuracy import Accuracy
from prompt_learner.optimizers.grid_search import GridSearch
import pandas as pd

In [ ]:
# Task description and allowed labels
TASK_DESCRIPTION = "You have to classify intent for customer messages sent to chatbot."
ALLOWED_LABELS = ['INFO_ADD_REMOVE_VEHICLE', 'INFO_LOGIN_ERROR',
       'INFO_ADD_REMOVE_INSURED', 'INFO_ERS', 'INFO_CAREERS',
       'INFO_DIFFERENT_AMTS', 'INFO_SPEAK_TO_REP', 'INFO_CANCEL_INS_POLICY',
       'INFO_UPDATE_LIENHOLDER', 'INFO_DELETE_DUPE_PYMT',
       'INFO_CANT_SEE_FARM_RANCH_POLICY', 'INFO_AUTO_INS_CANADA',
       'INFO_DEC_PAGE_NEEDED', 'INFO_LIFE_BENEFICIARY_CHANGE',
       'INFO_MAKE_PYMT']

In [ ]:
classification_task = ClassificationTask(description=TASK_DESCRIPTION,
                                         allowed_labels=ALLOWED_LABELS)

# Template for the task
template = MarkdownTemplate(task=classification_task)


In [ ]:
# Load training data
with open("train.csv") as f:
    for line in f:
        text, label = line.split(",")
        classification_task.add_example(Example(text=text.strip(), label=label.strip()))
#Add test examples
with open("test.csv") as f:
    for line in f:
        text, label = line.split(",")
        classification_task.add_example(Example(text=text.strip(),
                                                label=label.strip()),
                                                test=True)

In [ ]:
# Sample a random example from the training data
sampler = RandomSampler(num_samples=1, task=classification_task)
sampler.select_examples()

In [ ]:
# Assemble the prompt using the template
base_prompt = Prompt(template=template)
base_prompt.assemble_prompt()
print(base_prompt.prompt)


In [ ]:
# See performance on gpt3 turbo without any grid search
acc, results = Accuracy(classification_task).compute(base_prompt,
                                                     OllamaLocal(model_name='llama3'),
                                                     test=True)
print(acc)
print(pd.DataFrame(results))

In [ ]:
# Initialize a grid search on the current prompt
grid_search = GridSearch(prompt=base_prompt)
random_4_shot = RandomSampler(num_samples=4, task=classification_task)
random_15_shot = RandomSampler(num_samples=15, task=classification_task)
diverse_15_shot = DiverseSampler(num_samples=15, task=classification_task)
stratify_15_shot = StratifiedSampler(num_samples=1, task=classification_task)


In [ ]:
param_grid = {
    'sampler': [random_4_shot, random_15_shot,diverse_15_shot,stratify_15_shot],
    'template': [MarkdownTemplate, XmlTemplate],
    'adapter': [OllamaLocal(model_name='llama3')]#, Anthropic(model_name="claude-3-haiku-20240307"), OpenAI(model_name='gpt-4o')]
}

In [ ]:
# Perform grid search to find the best parameters
best_params, all_results = grid_search.search(param_grid)
print(pd.DataFrame(all_results))

In [ ]:
# Evaluate the best model
template = MarkdownTemplate(task=classification_task)
sampler = StratifiedSampler(num_samples=1, task=classification_task)
sampler.select_examples()

best_prompt = Prompt(template=template)
best_prompt.assemble_prompt()

In [ ]:
print(best_prompt.prompt)

In [ ]:
acc, results = Accuracy(classification_task).compute(best_prompt,
                                                     OllamaLocal(model_name = "llama3"),
                                                     test=True)
print(acc)
print(pd.DataFrame(results))

In [ ]:
best_prompt.add_inference("Paid the same thing twice return one please")

Anthropic().llm.invoke(best_prompt.prompt)